# Classification Analysis
This notebook will contain classification analysis for both the sensed and pipelined algorithms. Analysis will be preformed in regards for the sensed and pipelined algorthms themselves, as well as the ensemble algorithms. The analysis for the ensemble algorithm will focus on the HAMF android phones and the HAHF iOS phones.

## Dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import emeval.viz.geojson as ezgj

In [ ]:
# for analysized view
import emeval.analysed.phone_view as eapv

In [ ]:
import emeval.metrics.segmentation as ems

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import arrow
import numpy as np

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For maps
import folium
import branca.element as bre

In [ ]:
# For easier debugging while working on modules
import importlib

In [ ]:
import arrow

In [ ]:
def import_sd_and_pv_from_server(trips  = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"], 
                                 AUTHOR_EMAIL  = "shankari@eecs.berkeley.edu", 
                                 DATASTORE_LOC = "http://localhost:8080", 
                                 pkl_file_name = None):
    sd_l = []
    pv_l = []
    for trip in trips:
        sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, trip)
        pv = eipv.PhoneView(sd)
        sd_l.append(sd)
        pv_l.append(pv)
    if pkl_file_name:
        import pickle
        with open(pkl_file_name, 'wb') as outp:
            for pv in pv_l:
                pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)
    return sd_l, pv_l

In [ ]:
def import_pv_from_pkl(pkl_file_name, 
                       trips = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"]):
    import pickle
    pv_l = []
    with open('pv.pkl', 'rb') as inp:
        for trip in trips:
            pv_l.append(pickle.load(inp))
    return pv_l

In [ ]:
(pv_la, pv_sj, pv_ucb) = import_pv_from_pkl('pv.pkl')

### Get the sensed data for each trip

In [ ]:
%%capture
ems.fill_sensed_section_ranges(pv_la)
ems.fill_sensed_section_ranges(pv_sj)
ems.fill_sensed_section_ranges(pv_ucb)

## Get sensed timeline

```python
def get_trip_ss_and_gts_timeline(pv):
    """
    Get the sensed and ground truth timeline for each evaluation trip range for a given phone view.
    
    ----------
    Parameters
    ----------
    arg1: phone view
        A phone view to recieve timelines for.
    arg2: os
        a phone os to evaluate, must be one of 'ios' or 'android'
    arg3:
        an acuracy/frequency combination, must be one of 'accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'

    -------
    Returns
    -------
    list
        A list of trips for each phone view. Each trip has two entries, the sensed mode timeline and the ground truth timeline for the corresponding evaluation trip range.
    """
    ...
```

`TODO` break up the timelines by os and accuracy/frequency

In [ ]:
def get_trip_ss_and_gts_timeline(pv, os, role):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    trips = []
    for phone_os, phone_map in pv.map().items():
        if os != phone_os:
            continue
        for phone_label, phone_detail_map in phone_map.items():
            if "control" in phone_detail_map["role"]:
#                 print("Ignoring %s phone %s since they are always on" % (phone_detail_map["role"], phone_label))
                continue
            # this spec does not have any calibration ranges, but evaluation ranges are actually cooler
            for r in phone_detail_map["evaluation_ranges"]:
                if r['eval_role_base'] != role:
                    continue
                for tr in r["evaluation_trip_ranges"]:
                    tr_ss  = []
                    tr_gts = []
                    for ss in tr["sensed_section_ranges"]:
                        tr_ss.append(ss)
                    for section in tr["evaluation_section_ranges"]:
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr['trip_id_base'],
                                                                                  section['trip_id_base'],
                                                                                  tr['start_ts'],
                                                                                  tr['end_ts'])
                        if section_gt_leg["type"] == "WAITING":
#                             print("Skipping WAITING section %s %s with potential partway transitions" %
#                                   (tr["trip_id"], section["trip_id"]))
                            continue
                        # this calulcates the metric for the mode

                        ## and now we have the gt mode!
                        gts = {'start_ts': section['start_ts'], 
                               'end_ts': section['end_ts'], 
                               'mode': section_gt_leg['mode']}
                        tr_gts.append(gts)
                # now, we build a timeline for each trip
                trip = tr.copy()
                trip['ss_timeline']  = tr_ss
                trip['gts_timeline'] = tr_gts
                trips.append(trip)
    return trips

## Binary Classification (in seconds)
```python
def get_binary_class_in_sec(os, role, pv=None, pv_l=None):
    """
    This function computes binary classifications for a given set of trips in seconds.
    Using one unit of duration as our base unit, we calculate the following classifications:
        * True Positive
            + A true positive is when we sense that we are in a mode and we are in that mode.
        * False Positive
            + A false positive is when we sense that we are in a mode that we are not it.
        * False Negative
            + A false negative is when we are in a mode, but we do not sense being in that mode.
     
     Note that we compute the binary classifications for each sensed mode, but we combine the 'WALKING' and 'RUNNING' modes.
     Additionally, note that we have use the ground truth base mode when determining hits and misses. 
    
    ----------
    Parameters
    ----------
    arg1: string
        a phone os to evaluate, must be one of 'ios' or 'android'
    arg2: string.
        an acuracy/frequency combination, must be one of 'accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'.
    arg3: phone view
        A phone view to recieve timelines for, defaults to None.
    arg4: phone view list
        A phone view list to recieve timelines for, defaults to None.
        
    NOTE: must pass in either a phone view or a phone view list
    
    -------
    Returns
    -------
    list
        A list with the following entries
            [0] A dictionary of true positives with sensed modes as keys and TP hits has values.
            [1] A dictionary of false positives with sensed modes as keys and FP hits has values.
            [2] A dictionary of false negatives with sensed modes as keys and FN hits has values.
    
    """
    ...
```

In [ ]:
# taken from emission.core.wrapper.modeprediction
import enum as enum
class PredictedModeTypes(enum.Enum):
    UNKNOWN = 0
    WALKING = 1
    BICYCLING = 2
    BUS = 3
    TRAIN = 4
    CAR = 5
    AIR_OR_HSR = 6

In [ ]:
def get_binary_class_in_sec(os, role, pv=None, pv_l=None):
    BASE_MODE = {"WALKING": "WALKING",
                 "RUNNING" : "WALKING", 
                 "CYCLING" : "CYCLING",
                 "BICYCLING": "CYCLING",
                 "ESCOOTER": "CYCLING", 
                 "AUTOMOTIVE" : "AUTOMOTIVE",
                 "BUS": "AUTOMOTIVE",
                 "TRAIN": "TRAIN",
                 "LIGHT_RAIL": "TRAIN",
                 "SUBWAY": "TRAIN",
                 "CAR": "AUTOMOTIVE",
                 "AIR_OR_HSR": "TRAIN",
                 "INVALID" : "INVALID"}
    if pv:
        assert pv_l is None, "CANNOT PROVIDE BOTH PHONE VIEW AND PHONE VIEW LIST"
        trips = get_trip_ss_and_gts_timeline(pv, os, role)
    elif pv_l:
        trips = []
        for pv in pv_l:
            trips.extend(get_trip_ss_and_gts_timeline(pv, os, role))
    else:
        assert 1, "MUST PASS EITHER PHONE VIEW OR PHONE VIEW LIST"
    TP, FN, FP, TN = {}, {}, {}, {}
    for trip in trips:
        for mode in set(BASE_MODE.values()):
            for ss in trip['ss_timeline']:
                try:
                    # taken from emission.core.wrapper.modeprediction
                    ss = ss['data']
                    ss['mode'] = PredictedModeTypes(ss['sensed_mode'])._name_
                except:
                    pass
                ss_dur = ss['end_ts'] - ss['start_ts']
                gts_dur = 0
                for gts in trip['gts_timeline']:
                    if ss['end_ts'] >= gts['start_ts'] and ss['start_ts'] <= gts['end_ts']:
                        dur = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                        gts_dur += dur
                        if BASE_MODE[mode] == BASE_MODE[ss['mode']] and BASE_MODE[mode] == BASE_MODE[gts['mode']]:
                            TP[mode] = TP.setdefault(mode, 0) + dur
                        elif BASE_MODE[mode] == BASE_MODE[ss['mode']] and BASE_MODE[mode] != BASE_MODE[gts['mode']]:
                            FP[mode] = FP.setdefault(mode, 0) + dur
                        elif BASE_MODE[mode] != BASE_MODE[ss['mode']] and BASE_MODE[mode] == BASE_MODE[gts['mode']]:
                            FN[mode] = FN.setdefault(mode, 0) + dur
                        else:
                            TN[mode] = TN.setdefault(mode, 0) + dur
                leftover = ss_dur - gts_dur
                assert leftover >= 0, f"ERROR, NEGATIVE LEFTOVER OF {leftover}, NEED TO INVESTIGATE"
                if leftover > 0:
                    # invalid base mode maps to NO_GT mode
                    if mode == 'INVALID':
                        TP[mode] = TP.setdefault(mode, 0) + leftover
                    # We have no gts, but our modes are equal, so a false positive
                    elif BASE_MODE[mode] == BASE_MODE[ss['mode']]:
                        FP[mode] = FP.setdefault(mode, 0) + leftover
                    # We have no_gts, but our modes are unequal, so a true negative
                    else:
                        TN[mode] = TN.setdefault(mode, 0) + leftover
    return TP, FP, FN, TN

In [ ]:
%%capture
av_ucb =  eapv.create_analysed_view(pv_ucb, 'http://localhost:8080', "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

In [ ]:
df = pd.DataFrame(get_binary_class_in_sec('ios', 'HAHFDC', pv=pv_la))
dic={}
d = df.reset_index(drop=True)
dic['ios'+'\\_'+'HAHFDC'] = d
d = pd.concat(dic, axis=1)
d['Classifier'] = ['TP', 'FP', 'FN', 'TN']
print(d.set_index('Classifier').rename_axis(['Title', 'Mode'], axis=1).style.to_latex())

In [ ]:
df = pd.DataFrame(get_binary_class_in_sec('ios', 'HAMFDC', pv=pv_la))
dic={}
d = df.reset_index(drop=True)
dic['ios'+'\\_'+'HAMFDC'] = d
d = pd.concat(dic, axis=1)
d['Classifier'] = ['TP', 'FP', 'FN', 'TN']
print(d.set_index('Classifier').rename_axis(['Title', 'Mode'], axis=1).style.to_latex())

In [ ]:
df = pd.DataFrame(get_binary_class_in_sec('android', 'MAHFDC', pv=pv_la))
dic={}
d = df.reset_index(drop=True)
dic['ios'+'\\_'+'MAHFDC'] = d
d = pd.concat(dic, axis=1)
d['Classifier'] = ['TP', 'FP', 'FN', 'TN']
print(d.set_index('Classifier').rename_axis(['Title', 'Mode'], axis=1).style.to_latex())

# Get the error bars for trips

In [ ]:
def get_FN_and_FP_rate(spec_details):
    if type(spec_details) is not list: spec_details = [ spec_details ]
    av_l = []
#     for spec_detail in spec_details:
#         phone_view = eipv.PhoneView(spec_detail)
#         analysed_view = eapv.create_analysed_view(phone_view, spec_detail.DATASTORE_LOC, "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
#         av_l.append(analysed_view)
    for pv in [pv_la, pv_sj, pv_ucb]:
        analysed_view = eapv.create_analysed_view(pv, spec_details[0].DATASTORE_LOC, "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
        av_l.append(analysed_view)
    FN_rate = {'ios_HAHFDC' : {}, 'android_HAHFDC' : {}}
    FP_rate = {'ios_HAHFDC' : {}, 'android_HAHFDC' : {}}
    for analysed_view in av_l:
        (TP, FP, FN, TN) = get_binary_class_in_sec('ios', 'HAHFDC', pv=analysed_view)
        assert len(FN.keys()) == len(FP.keys()), f"FP: {FP.keys()}, \t FN: {FN.keys()}"
        for mode in FN.keys():
            FN_rate['ios_HAHFDC'][mode] = FN[mode] / (TP[mode] + FP[mode] + FN[mode] + TN[mode])
            FP_rate['ios_HAHFDC'][mode] = FN[mode] / (TP[mode] + FP[mode] + FN[mode] + TN[mode])
        (TP, FP, FN, TN) = get_binary_class_in_sec('android', 'HAHFDC', pv=analysed_view)
        assert len(FN.keys()) == len(FP.keys()), f"FP: {FP.keys()}, \t FN: {FN.keys()}"
        for mode in FN.keys():
            FN_rate['android_HAHFDC'][mode] = FN[mode] / (TP[mode] + FP[mode] + FN[mode] + TN[mode])
            FP_rate['android_HAHFDC'][mode] = FN[mode] / (TP[mode] + FP[mode] + FN[mode] + TN[mode])
    return FN_rate, FP_rate

In [ ]:
sd = eisd.ServerSpecDetails('http://localhost:8080', "shankari@eecs.berkeley.edu", "unimodal_trip_car_bike_mtv_la")
err = get_FN_and_FP_rate(sd)

In [ ]:
import json
print(json.dumps(err))

# $F_\beta$ score
$$
F_\beta = \frac {(1 + \beta^2) \cdot \mathrm{true\ positive} }{(1 + \beta^2) \cdot \mathrm{true\ positive} + \beta^2 \cdot \mathrm{false\ negative} + \mathrm{false\ positive}}
$$

```python
def get_F_score(pv, os, role, beta=1):
    """
    This function calculates the F score
    $$
    F_\beta = \frac {(1 + \beta^2) \cdot \mathrm{true\ positive} }{(1 + \beta^2) \cdot \mathrm{true\ positive} + \beta^2 \cdot \mathrm{false\ negative} + \mathrm{false\ positive}}
    $$
    based off data from a given set of phone views. Calls the get binary classification function
    
    ----------
    Parameters
    ----------
    arg1: phone view
        A phone view to recieve timelines for.
    arg2: os
        a phone os to evaluate, must be one of 'ios' or 'android'
    arg3:
        an acuracy/frequency combination, must be one of 'accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'
    arg4: int
        The beta value in which to use in the $F_\beta$ score. Defaults to 1.
        
    -------
    Returns
    -------
    dict:
        A dictionary with sensed modes as the keys and the corresponding of $F_\beta$ scores as the values.
    
    """
    ...
```

In [ ]:
def get_F_score(os, role, beta=1, pv=None, pv_l=None):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    if pv:
        assert pv_l is None, "CANNOT PROVIDE BOTH PHONE VIEW AND PHONE VIEW LIST"
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv=pv)
    elif pv_l:
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv_l=pv_l)
    else:
        assert 1, "MUST PASS EITHER PHONE VIEW OR PHONE VIEW LIST"
    F_score = {}
    for mode in TP.keys():
        numerator   = (1 + beta**2) * TP.setdefault(mode, 0)
        denominator = (1+beta**2) * TP.setdefault(mode, 0) + beta**2*FN.setdefault(mode, 0) + FP.setdefault(mode, 0)
        F_score[mode] = (numerator)/(denominator)
    return F_score

$$
MCC = \frac{TP \times TN - FP \times FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}
$$

In [ ]:
def get_mcc(os, role, beta=1, pv=None, pv_l=None):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    if pv:
        assert pv_l is None, "CANNOT PROVIDE BOTH PHONE VIEW AND PHONE VIEW LIST"
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv=pv)
    elif pv_l:
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv_l=pv_l)
    else:
        assert 1, "MUST PASS EITHER PHONE VIEW OR PHONE VIEW LIST"
    mcc = {}
    for mode in TP.keys():
        numerator   = TP.setdefault(mode, 0) * TN.setdefault(mode, 0) - FP.setdefault(mode, 0) * FN.setdefault(mode, 0)
        denominator_squared = (TP.setdefault(mode, 0) + FP.setdefault(mode, 0))*(TP.setdefault(mode, 0) + FN.setdefault(mode, 0))*(TN.setdefault(mode, 0) + FP.setdefault(mode, 0))*(TN.setdefault(mode, 0) + FN.setdefault(mode, 0))
        mcc[mode] = (numerator)/(np.sqrt(denominator_squared))
    return mcc

Informedness
$$
\frac{TP}{TP+FN} + \frac{TN}{TN+FP} -1
$$

In [ ]:
def get_inform(os, role, beta=1, pv=None, pv_l=None):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    if pv:
        assert pv_l is None, "CANNOT PROVIDE BOTH PHONE VIEW AND PHONE VIEW LIST"
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv=pv)
    elif pv_l:
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv_l=pv_l)
    else:
        assert 1, "MUST PASS EITHER PHONE VIEW OR PHONE VIEW LIST"
    info = {}
    for mode in TP.keys():
        n1 = TP.setdefault(mode, 0)
        d1 = TP.setdefault(mode, 0) + FN.setdefault(mode, 0)
        n2 = TN.setdefault(mode, 0)
        d2 = TN.setdefault(mode, 0) + FP.setdefault(mode, 0)
        info[mode] = (n1)/(d1)+(n2)/(d2) - 1
    return info

Accuracy
$$
\frac{TP + TN}{TP + TN+FP+FN}
$$

In [ ]:
def get_acc(os, role, beta=1, pv=None, pv_l=None):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    if pv:
        assert pv_l is None, "CANNOT PROVIDE BOTH PHONE VIEW AND PHONE VIEW LIST"
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv=pv)
    elif pv_l:
        (TP, FP, FN, TN) = get_binary_class_in_sec(os, role, pv_l=pv_l)
    else:
        assert 1, "MUST PASS EITHER PHONE VIEW OR PHONE VIEW LIST"
    acc = {}
    for mode in TP.keys():
        n1 = TP.setdefault(mode, 0) + TN.setdefault(mode, 0)
        d1 = TP.setdefault(mode, 0) + TN.setdefault(mode, 0) + FP.setdefault(mode, 0) + FN.setdefault(mode, 0)
        acc[mode] = (n1)/(d1)
    return acc

In [ ]:
def display_metrics(os, role, pv=None, pv_l=None):
    d = None
    for i, role in enumerate(['HAHFDC','HAMFDC', 'MAHFDC']):
        beta = 1
        df = pd.DataFrame(get_acc(os, role, pv=pv, pv_l=pv_l), index=[f"Accuracy"])
        df = pd.concat((df,
                        pd.DataFrame(get_F_score(os, role, pv=pv, pv_l=pv_l, beta=beta), index=[f"$F_{beta}$ score"])
                       ), axis=0 )
        df = pd.concat((df,
                        pd.DataFrame(get_mcc(os, role, pv=pv, pv_l=pv_l), index=[f"MCC score"])
                       ), axis=0 )
        df = pd.concat((df,
                        pd.DataFrame(get_inform(os, role, pv=pv, pv_l=pv_l), index=[f"Information score"])
                       ), axis=0 )
        if d is None:
            dic={}
            d = df.reset_index(drop=True)
            dic[role] = d
            d = pd.concat(dic, axis=1)
        else: 
            d = pd.concat((d,df.reset_index(drop=True)), axis=1)
            d = df.reset_index(drop=True)
            dic[role] = d
            d = pd.concat(dic, axis=1)
    d['Metrics'] = ['Accuracy', f"$F_{beta}$ score", f"MCC score", f"Information score"]
    return d.set_index('Metrics').rename_axis(['Role', 'Mode'], axis=1)

In [ ]:
df = display_metrics('ios', 'HAHFDC', pv_la)
df

In [ ]:
df = display_metrics('ios', 'HAHFDC', pv_la)
s = df.style
s
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000066; color: white;'
}
s.set_table_styles([cell_hover, index_names, headers])


s.set_table_styles({
    ('HAHFDC', 'WALKING'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)
s.set_table_styles({
    ('HAMFDC', 'WALKING'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)
s.set_table_styles({
    ('MAHFDC', 'WALKING'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)
s.set_table_styles([
    {'selector': 'th.col_heading', 'props': 'text-align: center;'},
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.5em;'},
    {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
], overwrite=False);s
# print(df.to_latex(index=False))

## Confusion Matrix
We will now generate confusion matrices based off OS and role, with the acctual modes as the rows, the predicted modes as the columns, and the entries as the base unit for the duration measurement

In [ ]:
def get_confusion_matrix(pv, os, role):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    cm_l = []
    trips = get_trip_ss_and_gts_timeline(pv, os, role)
    for trip in trips:
        for ss in trip['ss_timeline']:
            ss_dur = ss['end_ts'] - ss['start_ts']
            gts_dur = 0
            cm = {}
            for gts in trip['gts_timeline']:
                if ss['end_ts'] >= gts['start_ts'] and ss['start_ts'] <= gts['end_ts']:
                    dur = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                    gts_dur += dur
                    cm[gts['mode']] = cm.setdefault(gts['mode'], 0) + dur
            leftover = ss_dur - gts_dur
            assert leftover >= 0, f"ERROR, NEGATIVE LEFTOVER OF {leftover}, NEED TO INVESTIGATE"
            cm['NO_GT'] = cm.setdefault('NO_GT', 0) + leftover
            cm['sensed_mode'] = ss['mode']
            cm_l.append(cm)
    return cm_l

`TODO` make sure we can get confusion matrix accross phone views

In [ ]:
import itertools
def plot_confusion_matrix(os, pv=None, pv_l=None):
    assert pv != pv_l, "MUST PROVIDE EITHER PHONE VIEW OR PHONE VIEW LIST"
    assert pv is not None or pv_l is not None, "MUST PROVIDE EITHER PHONE VIEW OR PHONE VIEW LIST"
    if pv:
        title =  f"Confusion Matrix in seconds for {os}\n Phone View{pv.spec_details.CURR_SPEC_ID}"
    if pv_l:
        s=''
        for pv in pv_l:
            s = s + pv.spec_details.CURR_SPEC_ID + ',  '
        title =  f"Confusion Matrix in seconds for {os}\n Phone Views={s[:-3]}"
    fig, ax = plt.subplots(1,3, figsize=(24,14), dpi=300)
#     fig.suptitle(title, fontsize='xx-large')
    fig.text(0.5, 0.04, 'Predicted Label', ha='center', fontsize='xx-large')
    fig.text(0.04, 0.5, 'True Label', va='center', rotation='vertical', fontsize='xx-large')
    for k, role in enumerate(["HAHFDC", "HAMFDC", "MAHFDC"]):

        df = pd.DataFrame(get_confusion_matrix(pv, os, role)).groupby('sensed_mode').sum()    
        cm = ax[k].imshow(df.transpose(), interpolation='nearest',  cmap=plt.cm.Blues)
        ax[k].set_title(role)
    #     plt.colorbar(cm, ax=ax[0])
        tick_marks = np.arange(len(df))
        ax[k].set_yticks(np.arange(len(df.columns)))
        ax[k].set_xticks(np.arange(len(df)))
        ax[k].set_yticklabels(df)
        ax[k].set_xticklabels(df.index, rotation=80)
        color_thresh = df.max().max() / 2
        for i, j in itertools.product(range(df.shape[1]), range(df.shape[0])  ):
            ax[k].text(j, i, round(df.transpose().iat[i,j], 2), horizontalalignment='center', 
                   color='white' if df.transpose().iat[i,j] > color_thresh else 'black')
    #     break
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(cm, cax=cbar_ax)
    return fig

`TODO` plot andoid and ios for all the different a/f configurations

In [ ]:
plot_confusion_matrix('ios', pv_l=[pv_la, pv_sj, pv_ucb]).savefig('ios_cm.png')

In [ ]:
plot_confusion_matrix('android', pv_l=[pv_la, pv_sj, pv_ucb]).savefig('android_cm.png')

## Analyzed Data

In [ ]:
import emeval.analysed.phone_view as eapv

In [ ]:
importlib.reload(eapv)

In [ ]:
%%capture
av_la_clean   = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")
av_sj_clean   = eapv.create_analysed_view(pv_sj, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")
av_ucb_clean  = eapv.create_analysed_view(pv_ucb, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")

In [ ]:
plot_confusion_matrix('ios', pv=av_la_clean)

In [ ]:
get_binary_class_in_sec('ios', 'HAHFDC', pv=av_la_clean)